In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Confidence in hypothesis

This notebook demonsrtrates the evolution of confidence in a hypothesis. Depending on the hypothesis configuration of a subgraph we compare it to the maximum confidence we can obtain from a bigger graph.

## Statistics of the hypothesis graph

Number of nodes and arcs of the hypothesis graph

In [3]:
from hypotest.setup_hypothgraph import sample_graphs

INFO:rdflib:RDFLib Version: 4.2.1


In [4]:
hypothgraph = sample_graphs.sample_hypothgraph()

In [5]:
nb_nodes = len(hypothgraph.nodes())
nb_arcs = len(hypothgraph.edges())

In [6]:
print("number of nodes {}, number of arcs {}".format(nb_nodes, nb_arcs))

number of nodes 37, number of arcs 47


## Hypothesis causal endpoints

Creation of a subgraph is a bit of *woodo* magic, we need to say what is the desired ratio of the paths from *source* to *target*, the causal endpoints of a hypothesis. So before we proceed we need the hypothesis configuration.

In [7]:
import networkx as nx

In [8]:
from hypotest.graph_generation import hypoth_conf

In [9]:
source, target = hypoth_conf.generate_rich_endpoints(hypothgraph)

In [10]:
print('number of paths from {} to {} is {}'.format(source, target, len(list(nx.all_simple_paths(hypothgraph, source, target)))))

number of paths from http://plumdeq.xyz/ontologies/hypothesis/Mechanical_overloading to http://plumdeq.xyz/ontologies/hypothesis/MMP13_production is 3


## Create a sub hypothesis graph

We deliberately choose a very small ratio of `on_boundary` paths, however we take some fair amount of paths within the hypothesis configuration.

In [11]:
from hypotest.graph_generation import sub_hypothgraph

In [12]:
ratio_on_boundary_paths = 0.01
ratio_endpoints_paths = 0.2

In [13]:
subgraph = sub_hypothgraph.generate_sub_hypothgraph(
    hypothgraph, source, target,
    ratio_endpoints_paths=ratio_endpoints_paths,
    ratio_on_boundary_paths=ratio_on_boundary_paths)

In [14]:
nb_sub_nodes = len(subgraph.nodes())
nb_sub_arcs = len(subgraph.edges())

In [15]:
print("number of nodes {}, number of arcs {}".format(nb_sub_nodes, nb_sub_arcs))

number of nodes 17, number of arcs 20


## Computing confidences

To compute the confidences in both sub and hypothgraphs we need to determine what are boundary interiors in both of the graphs.

### Confidence computation

Confidence is computed as the sum of all weighted paths from `source` to `target` normalized to the number of these weighted paths. Suppose `source` is $u$ and `target` is $v$.

$$
Confidence(u, v) = \frac{\sum_{\pi_i(u, v)} weighted\_path(\pi_i)}{len(\pi(u, v))}
$$

### Boundary interiors of the two graphs

In [16]:
from hypotest.graph_generation import boundary

In [17]:
sub_boundary_interior = list(boundary.in_boundary_interior(subgraph, source, target))

In [18]:
big_boundary_interior = list(boundary.in_boundary_interior(hypothgraph, source, target))

In [19]:
print("Nb nodes in the boundary interior (subgraph) {}), nb nodes in full boundary interior {}".format(
    len(sub_boundary_interior), len(big_boundary_interior)))

Nb nodes in the boundary interior (subgraph) 7), nb nodes in full boundary interior 7


### Confidence subgraph

In [20]:
from hypotest.stats import confidences

Confidence in the hypothesis as we gradually add more evidenced nodes from the boundary, inside the subgraph only

In [21]:
sub_confidences = confidences.confidence_spectrum(subgraph, source, target)
sub_confidences

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

In [ ]:
sub_confidences_normalized = confidences.confidence_spectrum(subgraph, source, target, normalized=True)
sub_confidences_normalized

[0.0,
 0.14285714285714285,
 0.2857142857142857,
 0.42857142857142855,
 0.5714285714285714,
 0.7142857142857143,
 0.8571428571428571,
 1.0]

In [ ]:
plt.plot(sub_confidences_normalized)
plt.plot(sub_confidences)

/Users/asan/.Virtualenvs/hypothtest/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Confidence bigraph 

In [ ]:
big_confidences = confidences.confidence_spectrum(hypothgraph, source, target)
big_confidences

In [ ]:
big_confidences_normalized = confidences.confidence_spectrum(hypothgraph, source, target, normalized=True)
big_confidences_normalized

In [ ]:
plt.plot(big_confidences_normalized)
plt.plot(big_confidences)

## Relative confidence values

Here we check confidence values which can be obtained withing the small graph, normalized to the max confidence inside the small graph, as well as normalized to the max in the big graph

In [ ]:
import pandas as pd

In [ ]:
relative_spectrum = confidences.relative_confidence_spectrum(hypothgraph, subgraph, source, target)

In [ ]:
relative_spectrum_df = pd.DataFrame(relative_spectrum)

In [ ]:
relative_spectrum_df

## Draw this configuration

In [ ]:
from hypotest.io import write_dot

In [ ]:
current_conf = './images/subgraphs/current_conf.dot'

In [ ]:
with open(current_conf, 'w') as f:
    write_dot.to_dot(hypothgraph, subgraph, source, target, stream=f)